# Screencaps Web scraper

## from https://movie-screencaps.com/movie-directory/

### Function to grab all titles
### Function to grab URLs using title function
### Scrape images by looping over every frame and:
 - Extracting faces using MTCNN Framework 
 - Keep track of frame number
 - Save extracted faces as np arrays

*Ensure the frame number has not exceeded what exists by investigating response status code*

In [228]:
import requests
import httplib2
from bs4 import BeautifulSoup, SoupStrainer

In [182]:
url = "https://movie-screencaps.com/movie-directory/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [214]:
response.status_code

200

In [183]:
def get_movies():
    titles = []
    urls = []
    for column_section in soup.find_all("ul", class_="links"):
         for movie in column_section.find_all("li"):
                title = movie.find("a").string.strip()
                url_1 = movie.find(href=True)
                titles.append(title)
                urls.append(url_1["href"].strip())
    movies = dict(zip(titles, urls))
    return movies
                

In [192]:
movies = get_movies()

url_2 = movies['The A-Team (2010)']
#response_2 = requests.get(url_2)
#soup_2 = BeautifulSoup(response_2.content, "html.parser")

In [188]:
movies.items()

dict_items([('The A-Team (2010)', 'https://movie-screencaps.com/the-a-team-2010/'), ('Abbott and Costello Meet Frankenstein (1948)', 'https://movie-screencaps.com/abbott-costello-meet-frankenstein-1948/'), ('Abbott and Costello Meet the Mummy (1955)', 'https://movie-screencaps.com/abbott-and-costello-meet-the-mummy-1955/'), ('Abominable (2006)', 'https://movie-screencaps.com/abominable-2006/'), ('Ace Ventura: Pet Detective (1994)', 'https://movie-screencaps.com/ace-ventura-pet-detective-1994/'), ('Ace Ventura: When Nature Calls (1995)', 'https://movie-screencaps.com/ace-ventura-when-nature-calls-1995/'), ('The Addams Family (1991)', 'https://movie-screencaps.com/addams-family-1991/'), ('Air Bud (1997)', 'https://movie-screencaps.com/air-bud-1997/'), ('Air Bud 3: World Pup (2001)', 'https://movie-screencaps.com/air-bud-3-world-pup-2001/'), ('Air Bud: Golden Receiver (1998)', 'https://movie-screencaps.com/air-bud-golden-receiver-1998/'), ('Air Bud: Seventh Inning Fetch (2002)', 'https://

In [229]:
type(movies)

dict

In [164]:
soup_2.contents

['html',
 '\n',
 '[if lt IE 7]>\n<html class="lt-ie9 lt-ie8 lt-ie7" lang="en-US"> <![endif]',
 '\n',
 '[if IE 7]>\n<html class="lt-ie9 lt-ie8" lang="en-US"> <![endif]',
 '\n',
 '[if IE 8]>\n<html class="lt-ie9" lang="en-US"> <![endif]',
 '\n',
 '[if IE 9]>\n<html class="ie9" lang="en-US"> <![endif]',
 '\n',
 '[if gt IE 9]><!',
 '\n',
 <html lang="en-US"> <!--<![endif]-->
 <head>
 <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
 <meta content="True" name="HandheldFriendly"/>
 <meta content="yes" name="apple-touch-fullscreen">
 <meta content="320" name="MobileOptimized"/>
 <title>The A-Team (2010) - Movie- Screencaps.com</title>
 <link href="https://gmpg.org/xfn/11" rel="profile"/>
 <link href="https://movie-screencaps.com/xmlrpc.php" rel="pingback"/>
 <link crossorigin="" href="https://fonts.googleapis.com" 

In [197]:
#test out 

txt = 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-1.jpg'

x = txt.split("com-")

x[1].replace(f"{x[1]}", f"com-{2}")

'com-2'

In [250]:
def grab_frame_url(title):
    movies = get_movies()
    url = movies[title]
    http = httplib2.Http()
    status, response = http.request(url)
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            if '.jpg' in link['href']:
                frame_url = link['href']
                return frame_url

In [254]:
%time
grab_frame_url('The A-Team (2010)')

CPU times: user 11 µs, sys: 17 µs, total: 28 µs
Wall time: 66 µs


'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-1.jpg'

In [272]:
# create 3-column dataframe with movie title, frame number, and frame image arrays

In [271]:
from tqdm import tqdm 
from PIL import Image
import numpy as np
import pandas as pd

In [257]:
%time
img_url = 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-1.jpg'
img_response = requests.get(img_url, stream=True)

image = np.array(Image.open(img_response.raw))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


*backup function*
def get_images(title, frame_count):
    lst = []
    frame_url = grab_frame_url(title)
    for i in tqdm(range(1,frame_count,4)):
        split_url = frame_url.split("com-")
        new_frame = split_url[1].replace(f"{split_url[1]}", f"com-{i}.jpg")
        new_url = f"{split_url[0]}{new_frame}"
        response = requests.get(new_url, stream=True)
        if response.status_code != 200:
            break
        #preprocessing: resizing/compression, filtering of only frames with faces
        image = np.array(Image.open(response.raw))
        lst.append(image)
    return lst
        

In [275]:
img_lst = [1]
frame_lst = [1]
title_lst = ['hi']

img_dict = {'title':title_lst, 'frame_no':frame_lst, 'img_array':img_lst}
img_dict

{'title': ['hi'], 'frame_no': [1], 'img_array': [1]}

In [281]:
def get_images(title, frame_interval=2):
    img_lst = []
    frame_lst = []
    title_lst = []
    frame_url = grab_frame_url(title)
    i = 1
    while True:
        split_url = frame_url.split("com-")
        new_frame = split_url[1].replace(f"{split_url[1]}", f"com-{i}.jpg")
        new_url = f"{split_url[0]}{new_frame}"
        response = requests.get(new_url, stream=True)
        if response.status_code != 200:
            break
        image = np.array(Image.open(response.raw))
        img_lst.append(image)
        frame_lst.append(i)
        title_lst.append(title)
        print(i)
        i = i + frame_interval
    img_dict = {'title':title_lst, 'frame_no':frame_lst, 'img_array':img_lst}
    return pd.DataFrame.from_dict(img_dict)

In [282]:
%time
df = get_images('The A-Team (2010)', frame_interval=10)

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.9 µs
1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811
821
831
841
851
861
871
881
891
901
911
921
931
941
951
961
971
981
991
1001
1011
1021
1031
1041
1051
1061
1071
1081
1091
1101
1111
1121
1131
1141
1151
1161
1171
1181
1191
1201
1211
1221
1231
1241
1251
1261
1271
1281
1291
1301
1311
1321
1331
1341
1351
1361
1371
1381
1391
1401
1411
1421
1431
1441
1451
1461
1471
1481
1491
1501
1511
1521
1531
1541
1551
1561
1571
1581
1591
1601
1611
1621
1631
1641
1651
1661
1671
1681
1691
1701
1711
1721
1731
1741
1751
1761
1771
1781
1791
1801
1811
1821
1831
1841
1851
1861
1871
1881
1891
1901
1911
1921
1931
1941
1951
1961
1971
1981
1991
2001
2011
2021
2031
2041
2051
2061
2071
2081
20

In [285]:
df.to_csv('../raw_data/draft_df_ateam.csv')

In [280]:
df['img_array'][0].shape

(816, 1920, 3)

In [251]:
%time
get_frames('The A-Team (2010)')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


100%|██████████████████████████████████████████████| 1000/1000 [02:22<00:00,  7.02it/s]


['https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-1.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-5.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-9.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-13.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-17.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-21.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-25.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-29.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-33.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-37.jpg',
 'https://i1.wp.com/caps.pictures/201/0-theateam/full/ateam-movie-screencaps.com-41.jpg',
 'https://i1.

In [270]:
%time
results = get_images('The A-Team (2010)', 20000)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 8.82 µs


 28%|████████████▊                                 | 1397/5000 [07:04<18:13,  3.29it/s]


KeyboardInterrupt: 

In [269]:
len(results)

100